In [6]:
import polars as pl 

### Read data and aggregate by uf

In [7]:
data = pl.read_csv('../data/1_raw/dengue.csv.gz')
data = data.with_columns(
    pl.col('date').str.to_date().alias('date')
)
data = data.group_by(['date','epiweek','uf','train_1', 'target_1', 'train_2', 'target_2', 'train_3', 'target_3']).agg(
    pl.col('casos').sum().alias('casos'),
)

enso = pl.read_parquet('../data/2_inter/enso_interpolated.parquet')
geo_uf = pl.read_parquet('../data/2_inter/geodata_uf.parquet')

### Aggregate data

In [8]:
data = data.join(enso[['date','enso']], on='date', how='left').join(geo_uf, on='uf', how='left')

In [9]:
data.write_parquet('../data/3_primary/complete_dataset.parquet')